## Setup

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from tqdm import tqdm


In [ ]:
torch.manual_seed(42)
np.random.seed(42)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device

## Dataset Loading

### Load train and test datasets

DO NOT modify the cell below, this will load our dataset

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True
)

### Create DataLoaders

In [1]:
BATCH_SIZE = None # <- This is a hyperparameter for you to set

In [ ]:
train_dl = DataLoader(train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=0
)

I recommend leaving the test dataloader without shuffling and batch size of 1 to ensure we can reproduce results 

In [ ]:
test_dl = DataLoader(test_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=0
)

### Define tranforms that will happen to images

In [ ]:
"""
TODO: Define appropriate data transforms for training and validation

Hints:
- Use data augmentation for training (RandomHorizontalFlip, RandomRotation, etc.)
- Think if you want to do some normalisation...?
"""

 # YOUR CODE HERE - Define training transforms with augmentation
training_transform = transforms.Compose([
    # Add your transforms here
    transforms.ToTensor(),
])

# Do you want to add tranforms to the test dataset? Have a think about it
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

Below we apply the transforms that you defined (if any)

In [ ]:
train_dataset.transform = training_transform
test_dataset.transform = test_transform

### Visualise test data

Lets see what classes we have

In [ ]:
classes = test_dataset.classes
classes

In [ ]:
N_IMAGES = 10

fig, ax = plt.subplots(1, N_IMAGES, figsize=(17,7))

for i in range(N_IMAGES):
    im, lbl = test_dataset[i]
    
    # Convert from tensor (C, H, W) to numpy (H, W, C) for matplotlib
    im_display = im.permute(1, 2, 0)  # Change from (3, 32, 32) to (32, 32, 3)
    
    ax[i].imshow(im_display)
    ax[i].set_title(f'{classes[lbl]}')

## Define the model

Define your model below. I've left the backbine and classifier structure but feel free to define the model however you like. Typically backbone will have the *feature learning* part if your CNN where as the classifier or head will have the feed forward neural network

Tips:
- Start with 3 input channels (RGB)
- Gradually increase feature maps (64 -> 128 -> etc...)
- Think about what kernel size and padding you will use in your `Conv2D` blocks. 3x3 convolutions with padding=1 is a good start imo
- Don't neglect `MaxPool2d`
- Think about `Dropout`

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.5):
        super(Model, self).__init__()
        
        # TODO: Backbone - Feature extraction layers
        self.backbone = None
        
        # TODO: Classifier head - Fully connected layers
        self.classifier = None
        
    
    def forward(self, x):
        # TODO: Define me
        pass

In [ ]:
model = Model().to(device)
model

## Define the training loop

You can follow my template structure below where I create a function `train_epoch` for a single forward pass and `train_model` which runs a complete training run for a given number of epochs. Alternatively, feel free to define the training loop however you like :)

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train the model for one epoch
    """
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    # YOUR CODE HERE - Implement training loop
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader, desc="Training")):
        # Move data to device
        # Zero gradients
        # Forward pass
        # Calculate loss
        # Backward pass
        # Update weights
        # Calculate accuracy
        pass
    
    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct_predictions / total_samples
    
    return epoch_loss, epoch_acc

You can optionally also create a `validate_epoch` method if you you created a **validation** dataset

In [ ]:
def train_model(model, train_loader, num_epochs=25, learning_rate=0.001):
    """
    Complete training pipeline
    
    Steps:
    1. Define loss function and optimizer
    2. Optional: Add learning rate scheduler
    3. Training loop for specified epochs
    4. Return trained model and training history
    """
    
    # TODO: Define loss function and optimizer. 
    criterion = None
    optimizer = None
    
    # Optional: Learning rate scheduler
    # scheduler = None
    
    # Training history -> You don't have to use this. This is used to accumulate stats about your training run
    train_losses, train_accs = [], []
    
    print("Starting training...")
    
    # Training loop
    # TODO: YOUR CODE HERE - Implement training loop
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        # Train
        # Optionally print some stats for an epoch
        # Optionally store history in train_lossess and train accs
        # Optioanlly, if using a scheduler, do something with it
        pass
    
    print(f"\nTraining completed! Final accuracy: {train_accs[-1]:.2f}%")
    
    return model, {
        'train_losses': train_losses,
        'train_accs': train_accs
    }

### Train the model

In [ ]:
model, result = train_model(model,train_dl)

#### Plot results

This is optional, if you have kept you statistics about traning and validation, you can plot it below

In [ ]:
# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# TODO: Finish me

## Test Evalution -> Do not modify the eval code

In [ ]:
def evaluate_final_model(model, test_loader):
    """
    Evaluate the final model on test data and print comprehensive metrics
    
    Args:
        model: Trained PyTorch model
        test_loader: DataLoader for test data
    """
    model.eval()  # Set model to evaluation mode
    
    all_predictions = []
    all_targets = []
    
    print("Evaluating model on test data...")
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        for data, targets in tqdm(test_loader, desc="Testing"):
            # Move data to device
            data, targets = data.to(device), targets.to(device)
            
            # Forward pass
            outputs = model(data)
            
            # Get predictions
            _, predicted = torch.max(outputs, 1)
            
            # Store predictions and targets
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    # Convert to numpy arrays
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    
    # Calculate metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')
    
    # Print results
    print("\n" + "="*50)
    print("FINAL MODEL EVALUATION RESULTS")
    print("="*50)
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print("="*50)
    
    # Print detailed classification report
    print("\nDetailed Classification Report:")
    print(classification_report(all_targets, all_predictions, target_names=classes))
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': all_predictions,
        'targets': all_targets
    }

Pass the model to the eval function to get the final result and run the cell below

In [ ]:
results = evaluate_final_model(model, test_dl)